Process model for constant velocity model from euler angles.

Using euler angle convention of zyx, (yaw, pitch, roll)

In [5]:
import numpy as np
import sympy
import symforce.symbolic as sf

In [6]:
def R_z(angle : float) -> np.array:
    R_z = np.array([[np.cos(angle), -np.sin(angle), 0],
                    [np.sin(angle),  np.cos(angle), 0],
                    [0, 0, 1]])
    return R_z

def R_y(angle: float)-> np.array :
    R_y = np.array([[np.cos(angle), 0, np.sin(angle)],
                    [0, 1, 0],
                    [-np.sin(angle), 0, np.cos(angle)]])
    return R_y

def R_x(angle: float) -> np.array:
    R_x = np.array([[1, 0 , 0],
                    [0, np.cos(angle), -np.sin(angle)],
                    [0, np.sin(angle), np.cos(angle)]])
    return R_x

def euler_angle_to_rot(yaw: float, pitch: float, roll: float):
    
    return R_z(yaw) @ R_y(pitch) @ R_x(roll)

# http://www.stengel.mycpanel.princeton.edu/Quaternions.pdf
def euler_angle_rates_to_ang_vel(yaw_rate:float, pitch_rate:float, roll_rate:float, yaw:float, pitch:float, roll:float) -> np.array:
    """
    converts euler angles rates to body axis rates. 
    """
    euler_angle_rate_vec = np.array([roll_rate, pitch_rate, yaw_rate]).reshape(3, -1)
    psi = yaw
    theta = pitch
    phi = roll
    rate_matrix = np.array([
        [1, 0, -np.sin(theta)],
        [0, np.cos(phi), np.cos(theta)*np.sin(phi)],
        [0, -np.sin(phi), np.cos(phi)*np.cos(theta)]
    ])
    
    ang_vel = rate_matrix @ euler_angle_rate_vec
    
    return ang_vel


def ang_vel_to_euler_angle_rates(omega : np.array, yaw: float, pitch:float, roll:float):
    """
    omega - body axis rates
    
    """
    psi = yaw
    theta = pitch
    phi = roll
    omega = omega.reshape(3, -1)
    rate_matrix = np.array([
        [1, np.sin(phi)*np.tan(theta), np.cos(phi)*np.tan(theta)],
        [0, np.cos(phi), -np.sin(phi)],
        [0, np.sin(phi)*np.sec(theta), np.cos(phi)*np.sec(theta)]
    ])
    euler_angle_rates = rate_matrix @ omega
    
    return euler_angle_rates

def orientation_error_propagation(yaw:float, pitch:float, roll:float, body_omega:np.array):
    wx, wy, wz = body_omega
    qx = roll
    qy = pitch
    qz = yaw
    cx = np.cos(qx)
    sx = np.sin(qx)
    ty = np.tan(qy)
    sy = np.sin(qy)
    cy = np.cos(qy)
    
    
    A = np.array([
        [0, (wz*cx + wy*sx)/cy**2, ty*(wy*cx - wz*sx)],
        [0, 0, wz*cx + wy*sx],
        [0, (sy*(wz*cx + wy*sx))/cy**2, (wy * cx - wz*sx)/cy]
    ])
    
    return A@body_omega.reshape(3, -1)

Alternatives to euler angles  is rotation matrix (or direction cosines) and quaternions to avoid singularities.


Rotation Matrices

In [7]:
def skew(vec: np.array):
    M = np.array([
        [0, -vec[2], vec[1]],
        [vec[2], 0, -vec[0]],
        [-vec[1], vec[0], 0],
    ])
    
    return M
    
def vee(vec: np.array):
    M = np.array([
        [0, -vec[2], vec[1]],
        [vec[2], 0, -vec[0]],
        [-vec[1], vec[0], 0],
    ])
    return M


def angular_vel_to_rotation_rates(R_curr: np.array, omega: np.array):
    """
    angular error 
    """
    R_dot = R_curr@skew(omega)
    
    return R_dot

def angle_axis_to_rotation_matrix(axis : np.array, rotation_angle: float):
    delta = rotation_angle
    a = axis.reshape(3, -1)
    
    R = np.cos(delta)*np.eye(3) + (1 - np.cos(delta)) * a @ a.T - np.sin(delta)*skew(a)
    return R


def quaternion_to_rotation_matrix(q : np.array):
    q3 = q[0:3].reshape(3, -1)
    q4 = q[3]
    
    R = (q4**2 - np.dot(q3,q3))* np.eye(3) + 2*q3@q3.T - 2*q4*skew(q3)


Quaternions

In [8]:

def is_quaternion(q: np.array):
    return (np.dot(q, q) == 1)


def angle_axis_to_quaternion(axis: np.array, rotation_angle:float):
    s = np.sin(rotation_angle/2)
    c = np.cos(rotation_angle/2)
    q = np.array(s*axis[0], s*axis[1], s*axis[2], c)
    return q

def rot_mat_to_quaternion(R: np.array):
    
    q4 = 0.5*np.sqrt(1 + R[0,0] + R[1, 1]  + R[2, 2])
    q1 = 0.25*(R[1, 2] - R[2, 1])/q4
    q2 = 0.25*(R[2, 0] - R[0, 2])/q4
    q3 = 0.25*(R[0, 1] - R[1, 0])/q4
    
    return np.array([q1, q2, q3, q4])    

def quaternion_to_euler_angle(q:np.array):
    q1, q2, q3, q4 = q
    roll = np.atan2(2*(q1*q4 + q2*q3), 1 - 2*(q1**2 + q2**22))
    pitch = np.asin(2*(q2*q4 - q1*q3))
    yaw = np.atan2(2*(q3*q4 + q1*q2), 1 - 2(q2**2 + q3**2))
    
    return roll, pitch, yaw

def angular_vel_to_quaternion_rates(ang_vel: np.array, q: np.array):
    
    q3 = q[0:3].reshape(3, -1)
    q4 = q[3]
    
    wx, wy, wz = ang_vel
    A = np.array([
        [0, wz, -wy, wx],
        [-wz, 0, wx, wy],
        [wy, -wx, 0, wz],
        [-wx, -wy, -wz, 0]
    ])
    
    q_dot = A @ q.reshape(4, -1)
    return q_dot



<>:24: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<>:24: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
/tmp/ipykernel_1187411/3918344096.py:24: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  yaw = np.atan2(2*(q3*q4 + q1*q2), 1 - 2(q2**2 + q3**2))
